# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: 

**Professor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
from pcamarillor.spark_utils import SparkUtils
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.functions import trim

log_schema = SparkUtils.generate_schema([
    ("timestamp", "string"),
    ("level", "string"),
    ("message", "string"),
    ("server", "string")
])

logs_df = spark.readStream \
    .format("csv") \
    .option("sep", "|") \
    .schema(log_schema) \
    .load("/opt/spark/work-dir/data/logs/")

logs_df = logs_df.select(
    trim(logs_df["timestamp"]).alias("timestamp"),
    trim(logs_df["level"]).alias("level"),
    trim(logs_df["message"]).alias("message"),
    trim(logs_df["server"]).alias("server")
)

critical_errors = logs_df.filter(logs_df["message"].contains("500"))

aggregated_errors = critical_errors.groupBy(
    "timestamp", "level", "message", "server"
).count()

query = aggregated_errors.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination(60)
